<a href="https://colab.research.google.com/github/venkateshpaturu/venkatesh-HDS/blob/main/midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mid-term for HDS5210

Your supervisor is concerned about 4-year survival risks for COPD. She has asked for you to do some analysis using a new metric, BODE. BODE is an improvement on a previous metric and promises to provide insight on survival risks.

BODE is defined here. https://www.mdcalc.com/calc/3916/bode-index-copd-survival#evidence

Your assignment is to create a BODE calculation, use it to calculate BODE scores and BODE survival rates for a group of patients. Then we want to evaluate the average BODE scores and BODE survival rates for each area hospital.

Your patient input file will have the following columns:
NAME,SSN,LANGUAGE,JOB,HEIGHT_M,WEIGHT_KG,fev_pct,dyspnea_description,distance_in_meters,hospital

BODE calculations require a BMI value, so you will have to create a function for it.

Your output should be in the form of two CSV files, patient_output.csv and hospital_output.csv.

Patient_output will have the following columns:
NAME,BODE_SCORE,BODE_RISK,HOSPITAL

Hospital output will have the following columns:
HOSPITAL_NAME, COPD_COUNT, PCT_OF_COPD_CASES_OVER_BEDS, AVG_SCORE, AVG_RISK

Each function you create should have documentation and a suitable number of test cases. If the input data could be wrong, make sure to raise a Value Error.

For this assignment, use the doctest, json, and csv libraries. Pandas is not allowed for this assignment.

In [32]:
import doctest
import json
import csv

### Step 1: Calculate BMI

In [33]:
def calculate_bmi(weight_kg, height_m):
    """
    Calculate BMI from weight and height.

    >>> calculate_bmi(70, 1.75)
    22.86
    >>> calculate_bmi(90, 1.90)
    24.93
    """
    if weight_kg <= 0 or height_m <= 0:
        raise ValueError("Weight and height must be positive values.")

    return round(weight_kg / (height_m ** 2), 2)

### Step 2: Calculate BODE Score

In [34]:
def calculate_bode_score(bmi, fev_pct, dyspnea, distance):
    """
    Calculate the BODE score based on BMI, FEV1%, dyspnea, and distance.
    Returns the BODE score as an integer.
    """
    # BMI score
    if bmi >= 21:
        bmi_score = 0
    elif 21 > bmi >= 21:
        bmi_score = 1
    else:
        bmi_score = 2

    # FEV1% score
    if fev_pct >= 65:
        fev_score = 0
    elif 50 <= fev_pct < 65:
        fev_score = 1
    elif 36 <= fev_pct < 50:
        fev_score = 2
    else:
        fev_score = 3

    # Dyspnea score
    dyspnea_scores = {
        'None': 0,
        'Mild': 1,
        'Moderate': 2,
        'Severe': 3,
    }
    dyspnea_score = dyspnea_scores.get(dyspnea, 0)  # Default to 0 if dyspnea is not in the dictionary

    # calculating the distance walked score
    if distance >= 350:
        distance_score = 0
    elif 250 <= distance < 350:
        distance_score = 1
    elif 150 <= distance < 250:
        distance_score = 2
    else:
        distance_score = 3

    # Total BODE score
    bode_score = bmi_score + fev_score + dyspnea_score + distance_score
    return bode_score


### Step 3: Calculate BODE Risk

In [35]:
def calculate_bode_risk(bode_score):
    """
    Calculating the 4-year survival risk based on the BODE score.
    Returns the survival risk percentage as a float.
    """
    # Defining the survival rates based on BODE score
    survival_risk_mapping = {
        0: 80.0,  # 80% 4-year survival for BODE score 0-2
        1: 80.0,
        2: 80.0,
        3: 67.0,  # 67% 4-year survival for BODE score 3-4
        4: 67.0,
        5: 57.0,  # 57% 4-year survival for BODE score 5-6
        6: 57.0,
        7: 35.0,  # 35% 4-year survival for BODE score 7-10
        8: 35.0,
        9: 35.0,
        10: 35.0
    }

    # Checking the bode_score is between 0 and 10
    bode_score = min(max(bode_score, 0), 10)

    # Returning the corresponding survival risk
    return survival_risk_mapping.get(bode_score, 35.0)


### Step 4: Load Hospital Data

In [36]:
import json

hospital_json = "hospitals.json"

# Load hospital data from the JSON file
with open(hospital_json, 'r') as f:
    data = json.load(f)

hospital_beds = {}
hospital_metrics = {}

# Looping through each system in the data
for system in data:
    # Each system that has  a list of hospitals
    if 'hospitals' in system:
        for hospital in system['hospitals']:
            hospital_name = hospital.get('name')
            beds = hospital.get('beds', 0)

            if hospital_name:
                hospital_beds[hospital_name] = beds

                # Initializing hospital metrics
                hospital_metrics[hospital_name] = {
                    'total_bode_score': 0,
                    'total_risk': 0,
                    'copd_count': 0,
                    'beds': beds
                }



### Step 5: Main business logic

Call BODE Score, BODE Risk functions for each patient.

For each hospital, calculate Avg BODE score and Avg BODE risk and count the number of cases for each hospital.

In [37]:



patient_csv = "patient.csv"
hospital_json_file = "hospitals.json"

patient_output_file = "patient_output.csv"
hospital_output_file = "hospital_output.csv"

patient_results = []

# Read the patient data from the CSV file
with open(patient_csv, 'r') as csvfile:
    reader = csv.DictReader(csvfile)

    for row in reader:
        name = row['NAME']
        ssn = row['SSN']
        language = row['LANGUAGE']
        job = row['JOB']
        height_m = float(row['HEIGHT_M'])
        weight_kg = float(row['WEIGHT_KG'])
        fev_pct = float(row['fev_pct'])
        dyspnea_description = row['dyspnea_description']
        distance_in_meters = float(row['distance_in_meters'])
        hospital_name = row['hospital']

        # Calculate BMI, BODE score, and BODE risk
        bmi = calculate_bmi(weight_kg, height_m)
        bode_score = calculate_bode_score(bmi, fev_pct, dyspnea_description, distance_in_meters)
        bode_risk = calculate_bode_risk(bode_score)

        # Add patient results
        patient_results.append([name, bode_score, bode_risk, hospital_name])

        # Update hospital metrics
        if hospital_name in hospital_metrics:
            hospital_metrics[hospital_name]['total_bode_score'] += bode_score
            hospital_metrics[hospital_name]['total_risk'] += bode_risk
            hospital_metrics[hospital_name]['copd_count'] += 1

hospital_output_list = []

# Calculate hospital metrics
for hospital_name, metrics in hospital_metrics.items():
    copd_count = metrics['copd_count']
    if copd_count > 0:
        avg_bode_score = metrics['total_bode_score'] / copd_count
        avg_bode_risk = metrics['total_risk'] / copd_count
    else:
        avg_bode_score = 0
        avg_bode_risk = 0
    pct_of_copd_cases = (copd_count / metrics['beds']) * 100 if metrics['beds'] > 0 else 0
    hospital_output_list.append([hospital_name, copd_count, pct_of_copd_cases, avg_bode_score, avg_bode_risk])

# Write patient_output.csv
with open(patient_output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["NAME", "BODE_SCORE", "BODE_RISK", "HOSPITAL"])
    writer.writerows(patient_results)

# Write hospital_output.csv
with open(hospital_output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["HOSPITAL_NAME", "COPD_COUNT", "PCT_OF_COPD_CASES_OVER_BEDS", "AVG_SCORE", "AVG_RISK"])
    writer.writerows(hospital_output_list)


In [38]:
if __name__ == "__main__":
    doctest.testmod()
